In [29]:
import csv
import pandas as pd
import sinling
from sinling import SinhalaTokenizer, POSTagger, SinhalaStemmer
from sinling import SinhalaTokenizer, POSTagger

In [12]:
# To find the most closed previous noun for the verb
def find_next_verb_after_target(pos_tags, target_index):
    next_verb_index = None
    for sentence_tags in pos_tags:
        for word_index, (word, pos_tag) in enumerate(sentence_tags):
            if word_index > target_index:
                if pos_tag.startswith('VFM') or pos_tag.startswith('VP') or pos_tag.startswith('POST') or pos_tag.startswith('VNN'):  
                    next_verb_index = word_index
                    break
    return next_verb_index

In [19]:
# Tokenize the document
def tokenize_document_and_return_POS_TAG(document):
    tokenizer = SinhalaTokenizer()
    tokenized_sentences = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(document)]
    tagger = POSTagger()
    pos_tags = tagger.predict(tokenized_sentences)
    return pos_tags

In [33]:
from sinling import SinhalaTokenizer, POSTagger
# document = 'ඔතන තියෙන දදය ගන්න පසුව පැමිණෙනවා'
document = 'අමර ඔතන දදය තබන්න ඉන්පසු ඔබ ගෙදරට යන්න'

tokenizer = SinhalaTokenizer()
tokenized_sentences = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(document)]
tagger = POSTagger()
pos_tags = tokenize_document_and_return_POS_TAG(document)



# Specify the target index you're interested in (e.g., index 5)
target_index = 6
result = find_next_verb_after_target(pos_tags, target_index)

if result is not None:
    print(f"The index of the first noun after the target index is: {result}")
else:
    print("No verb found before the target index.")


The index of the first noun after the target index is: 7


In [47]:
def search_noun_index(file_path, sentence):
    with open(file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)  # Assuming the first row is a header

        # Assuming the sentence is in the first column (index 0)
        sentence_column_index_1 = 0  # first column in the CSV file
        sentence_column_index_2 = 1  # second column in the CSV file
        replacement_column_index = 2  # third column in the CSV file

        # Split the sentence into words
        sentence_words = sentence.split()
        stem_sentence_words=convert_to_stam_wordsList(sentence)
        noun_index = set()
        x=len(sentence_words)
        for row in csv_reader:
            # Get the words from the CSV columns
            csv_column_word_1 = row[sentence_column_index_1]
            csv_column_word_2 = row[sentence_column_index_2]
            replacement_word = row[replacement_column_index]
            
            #check if original word have -- ගෙදරට
            if csv_column_word_2 in sentence_words:
                for index, word in enumerate(sentence_words):
                    if((x-1)>index):
                        if(csv_column_word_2==word):
                            noun_index.add(index)
            #check if stam word have  -- ගෙදර
            if csv_column_word_2 in stem_sentence_words:
                for index, word in enumerate(stem_sentence_words):
                    if((x-1)>index):
                        if(csv_column_word_2==word):
                            noun_index.add(index)
        
        return (noun_index)    
    
# csv_file_path = 'verb_list.csv'
# search_sentence = 'අමර ඔතන දදය තබන්න ඉන්පසු ඔබ ගෙදරට යන්න'
# verbindex=search_noun_index(csv_file_path, search_sentence)
# 
# print(verbindex)

{6}


In [48]:
#normal original words can be not in our db then the stam words can be have then to check both here it convert to stam list also
def convert_to_stam_wordsList(sentence_words):
    sentence_words = search_sentence.split()
    new_sentence_words=[]
    stemmer = SinhalaStemmer()
    for index, word in enumerate(sentence_words):    
        stemmed_word = stemmer.stem(word)
        # Accessing the first element
        first_element = stemmed_word[0]
        new_sentence_words.append(first_element)
    return (new_sentence_words)


In [39]:
search_sentence = 'අමර ඔතන දදය තබන්න ඉන්පසු ඔබ ගෙදරට යන්න'
print(convert_to_stam_wordsList(search_sentence))

['අමර', 'ඔත', 'දද', 'තබ', 'ඉන්පස', 'ඔබ', 'ගෙදර', 'ය']


In [50]:
#here we are going to replace next verb
def find_replace_verb(file_path, sentence,index):
    with open(file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)  # Assuming the first row is a header

        # Assuming the sentence is in the first column (index 0)
        sentence_column_index_1 = 0  # Second column in the CSV file
        sentence_column_index_2 = 1  # Third column in the CSV file
        replacement_column_index = 2  # Fourth column in the CSV file

        updated_sentence = sentence

        # Split the sentence into words
        sentence_words = sentence.split()
        pos_tags =tokenize_document_and_return_POS_TAG(sentence)
        result = find_next_verb_after_target(pos_tags, index)
        
        for row in csv_reader:
            # Get the words from the CSV columns
            csv_column_word_1 = row[sentence_column_index_1]
            csv_column_word_2 = row[sentence_column_index_2]
            replacement_word = row[replacement_column_index]
            
            stemmer = SinhalaStemmer()
            stemmed_word = stemmer.stem(sentence_words[index])
            first_element = stemmed_word[0]
            second_element=stemmed_word[1]
            updated_sentence=""
            # Check if the words from both columns are in the sentence
            
            if csv_column_word_1 == sentence_words[result] and csv_column_word_2 == sentence_words[index]:
                # print("first")
                # print(f"csv_column_word_1- {csv_column_word_1}")
                # print(f"sentence_words[result]- {sentence_words[result]}")
                # print(f"csv_column_word_2- {csv_column_word_2}")
                # print(f"sentence_words[index]- {sentence_words[index]}")
                updated_sentence = sentence.replace(csv_column_word_1, replacement_word)
                return updated_sentence
            elif csv_column_word_1 == sentence_words[result] and csv_column_word_2 == first_element:
                # print("second")
                # print(f"csv_column_word_1- {csv_column_word_1}")
                # print(f"sentence_words[result]- {sentence_words[result]}")
                # print(f"csv_column_word_2- {csv_column_word_2}")
                # print(f"sentence_words[index]- {sentence_words[index]}")
                # 
                # print(f"replacement_word- {replacement_word}")
                # print(f"second_element- {second_element}")
                new_replacement_word=replacement_word+second_element
                # print(f"new_replacement_word- {new_replacement_word}")
                
                updated_sentence = sentence.replace(sentence_words[result], replacement_word)
                # print(f"sentence- {sentence}")
                # print(f"updated_sentence- {updated_sentence}")
                return updated_sentence
        return sentence


In [55]:
# Example usage
csv_file_path = 'verb_list.csv'

# Test Case 1
search_sentence = 'අමර ඔතන දදය තබන්න ඉන්පසු ඔබ ගෙදරට යන්න'
update=find_replace_verb(csv_file_path, search_sentence,6)
print(f"previous sentence {search_sentence}")
print(f"final sentence {update}")

previous sentence අමර ඔතන දදය තබන්න ඉන්පසු ඔබ ගෙදරට යන්න
final sentence අමර ඔතන දදය තබන්න ඉන්පසු ඔබ ගෙදරට යනවා


In [58]:
# Example usage
csv_file_path = 'verb_list.csv'

search_sentence = 'අමර ඔතන කොඩිය ගන්න ඉන්පසු ඔබ ගෙදරට යන්න'
verbindex=search_noun_index(csv_file_path, search_sentence)

print(verbindex)
pos_tags =tokenize_document_and_return_POS_TAG(search_sentence)
update_sentence=search_sentence
i=1
for element in verbindex:
    print(f"{i}- iteration")
    update_sentence=find_replace_verb(csv_file_path, update_sentence,element)
    print(f"previous sentence {search_sentence}")
    print(f"final sentence {update_sentence}")
    i=i+1

{2, 6}
1- iteration
previous sentence අමර ඔතන කොඩිය ගන්න ඉන්පසු ඔබ ගෙදරට යන්න
final sentence අමර ඔතන කොඩිය ගන්නවා ඉන්පසු ඔබ ගෙදරට යන්න
2- iteration
previous sentence අමර ඔතන කොඩිය ගන්න ඉන්පසු ඔබ ගෙදරට යන්න
final sentence අමර ඔතන කොඩිය ගන්නවා ඉන්පසු ඔබ ගෙදරට යනවා


***For testing sections***

In [6]:
from sinling import SinhalaTokenizer, POSTagger

tokenizer = SinhalaTokenizer()

document = 'අමර ඔතන දදය තබන්න ඉන්පසු ඔබ ගෙදරට යන්න'  # may contain multiple sentences

tokenized_sentences = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(document)]

tagger = POSTagger()

pos_tags = tagger.predict(tokenized_sentences)

for sentence_tags in pos_tags:
    for word, pos_tag in sentence_tags:
        print(f"Word: {word}, POS Tag: {pos_tag}")


Word: අමර, POS Tag: NNC
Word: ඔතන, POS Tag: VP
Word: දදය, POS Tag: NNC
Word: තබන්න, POS Tag: VNN
Word: ඉන්පසු, POS Tag: POST
Word: ඔබ, POS Tag: PRP
Word: ගෙදරට, POS Tag: NNC
Word: යන්න, POS Tag: POST
Word: ., POS Tag: FS


VFM,VP.POST

In [34]:
from sinling import SinhalaTokenizer, POSTagger, SinhalaStemmer
stemmer = SinhalaStemmer()
stemmed_word = stemmer.stem("තබන්න")
print(stemmed_word)
# Accessing the first element
first_element = stemmed_word[0]
print("First Element:", first_element)

# Accessing the second element
second_element = stemmed_word[1]
print("Second Element:", second_element)

('තබ', 'න්න')
First Element: තබ
Second Element: න්න


In [6]:
sentence = "එය දුටු මහ සද්ද නිසා පැමිණි තොටියා,ගං මෝය සිට නැවත තොටුපළට පැමිණියේය. එවිට නිශාචර ජීවි දුටු තොටියා රාත්‍රිය පැමිණියේ බව දැක්කේය.ඉන්පසු අශ්වයකුගේ හේශාරාවය ඇසුනු විගස ඉක්මනට ගොඩට දිව ගියේය."